# Ray Tune Example

#### Examples from: 

* https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html
* https://docs.ray.io/en/latest/tune/index.html


In [1]:
import os
from collections import Counter
import platform
import time
import numpy as np

import ray
from ray import tune

from tune_sklearn import TuneGridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV


Package pickle5 becomes unnecessary in Python 3.8 and above. Its presence may confuse libraries including Ray. Please uninstall the package.


If we have a ray cluster to connect to, do it. otherwise rely on ray to manage distribution of work locally. 

In [2]:
os.environ['RAY_CLUSTER']

'ray-cluster-eerlands-40redhat-2ecom-ray-head'

In [3]:
ray.init('ray://{ray_head}:10001'.format(ray_head=os.environ['RAY_CLUSTER']))

ClientContext(dashboard_url='10.129.4.133:8265', python_version='3.8.12', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', protocol_version='2022-03-16', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7f805af11d90>)

In [4]:
# Create dataset
X, y = make_classification(
    n_samples=11000,
    n_features=1000,
    n_informative=50,
    n_redundant=0,
    n_classes=10,
    class_sep=2.5,
)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1000)

As a baseline to evaluate our increased performance. Let's run SGD once, with one set of parameters and see how long it takes. 

In [5]:
%%time
clf = SGDClassifier(alpha=1,epsilon=0.1)
clf.fit(x_train, y_train)
first_guess_score = clf.score(x_test, y_test)
first_guess_score

CPU times: user 1.59 s, sys: 7.93 ms, total: 1.6 s
Wall time: 1.59 s


0.869

Note that this objective function includes our training and testing data in its closure. Inefficient!

In [6]:
def objective(config, checkpoint_dir=None):
    clf = SGDClassifier(alpha = config["alpha"], epsilon = config["epsilon"])
    clf.fit(x_train,y_train)
    score = clf.score(x_test,y_test)
    return {"score": score}

define our new search space for the SGD (Stochastic Gradient Descent) algorithm. 

In [7]:
search_space = {
    "alpha": tune.grid_search([1e-4, 1e-1, 1]),
    "epsilon": tune.grid_search([0.01, 0.1]),
}

Looks like it takes about 1 second to initialize the SGD, train it and evaluate it. Give our search space contains 6 different combinations, we should expect a serial execution (slowest) to be about 6 seconds, and a fully parralelized implementation to take about 1 second (fastest). Lets see.    

In [8]:
%%time
analysis = tune.run(objective,
                    config=search_space,
                    resources_per_trial={'gpu': 1})

/opt/app-root/lib64/python3.8/site-packages/ray/util/client/worker.py:604: UserWarning: More than 10MB of messages have been created to schedule tasks on the server. This can be slow on Ray Client due to communication overhead over the network. If you're running many fine-grained tasks, consider running them inside a single remote function. See the section on "Too fine-grained tasks" in the Ray Design Patterns document for more details: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.f7ins22n6nyl. If your functions frequently use large objects, consider storing the objects remotely with ray.put. An example of this is shown in the "Closure capture of large / unserializable object" section of the Ray Design Patterns document, available here: https://docs.google.com/document/d/167rnnDFIVRhHhK4mznEIemOtj63IOhtIPvSYaPgI4Fg/edit#heading=h.1afmymq455wu
  warnings.warn(
(run pid=1517) 2022-08-16 23:28:32,202	ERROR syncer.py:147 -- Log sync require

(run pid=1517) == Status ==
(run pid=1517) Current time: 2022-08-16 23:28:34 (running for 00:00:08.92)
(run pid=1517) Memory usage on this node: 6.4/30.9 GiB
(run pid=1517) Using FIFO scheduling algorithm.
(run pid=1517) Resources requested: 1.0/4 CPUs, 1.0/1 GPUs, 0.0/4.2 GiB heap, 0.0/1.72 GiB objects
(run pid=1517) Result logdir: /home/ray_results/objective_2022-08-16_23-28-25
(run pid=1517) Number of trials: 6/6 (5 PENDING, 1 RUNNING)
(run pid=1517) +-----------------------+----------+-------------------+---------+-----------+
(run pid=1517) | Trial name            | status   | loc               |   alpha |   epsilon |
(run pid=1517) |-----------------------+----------+-------------------+---------+-----------|
(run pid=1517) | objective_20004_00000 | RUNNING  | 10.129.4.133:1713 |  0.0001 |      0.01 |
(run pid=1517) | objective_20004_00001 | PENDING  |                   |  0.1    |      0.01 |
(run pid=1517) | objective_20004_00002 | PENDING  |                   |  1      |      

(run pid=1517) 2022-08-16 23:29:42,908	INFO tune.py:747 -- Total run time: 79.01 seconds (77.14 seconds for the tuning loop).


In [9]:
best = analysis.get_best_config(metric="score", mode="max")
best

{'alpha': 0.1, 'epsilon': 0.01}

In [10]:
%%time
clf = SGDClassifier(**best)
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

CPU times: user 2.81 s, sys: 27.2 ms, total: 2.83 s
Wall time: 2.84 s


0.875

In [11]:
first_guess_score

0.869